## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [3]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2019-12-12 18:02:34--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2019-12-12 18:02:35--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8313bac95b9b805ff675fc52c5.dl.dropboxusercontent.com/cd/0/get/AuEytEEd1d1qu5fgJfymCg6HwnY-yOZbrTWZb2QJPzOFnlcrciHQXUxaMTmMyh-eHF4i1uXx6FD3TwjcZR4uNSTbvuJewolbZhWjeGqXCmFpXA/file?dl=1# [following]
--2019-12-12 18:02:35--  https://uc8313bac95b9b805ff675fc52c5.dl.dropboxusercontent.com/cd/0/get/AuEytEEd1d1qu5fgJfymCg6HwnY-yOZbrTWZb2QJPzOFnlcrciHQXUxaMTmMyh-eHF4i1uXx6FD3TwjcZR4uNSTbvuJewolbZhWjeGqXCmFpXA/file?dl=1
Resolving uc8313ba

In [4]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [5]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [6]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = []
for line in data:
  line_lower = []
  for token in line:
    line_lower.append(token.lower())
  data_tok.append(tokenizer.tokenize(''.join(line_lower)))
print(data_tok[50])

['what', 'tv', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [0]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [8]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [0]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [10]:
# now you can get word vectors !
model.get_vector('anything')

array([ 2.0502264 , -1.7658955 , -2.6428092 , -0.00874853,  0.8043227 ,
        2.134152  , -1.245339  , -2.4172938 ,  2.057879  , -2.5831647 ,
       -2.0694978 ,  0.4839197 ,  0.8694971 , -0.20812577,  2.582804  ,
        0.33915678, -1.6224458 , -1.4341171 , -1.5694226 , -1.0155085 ,
        1.6022162 ,  0.705603  ,  1.0642095 , -4.662064  ,  0.3948068 ,
        1.1540353 , -3.2533488 ,  2.2824712 , -3.3063736 , -1.5044583 ,
       -2.5066943 ,  2.836187  ], dtype=float32)

In [11]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('rice', 0.9540687799453735),
 ('fruit', 0.9298794865608215),
 ('cheese', 0.9241037964820862),
 ('corn', 0.9133297204971313),
 ('cream', 0.9079626798629761),
 ('grass', 0.9079277515411377),
 ('sugar', 0.90557861328125),
 ('garlic', 0.9047470092773438),
 ('potatoes', 0.9022581577301025),
 ('honey', 0.9010318517684937)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [12]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.538511335849762),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233934879303),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [14]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [15]:
print(words[:10])

['<user>', '.', ':', 'rt', ',', '<repeat>', '<hashtag>', '<number>', '<url>', '!']


In [16]:
# for each word, compute its vector with model
word_vectors = []
for word in words:
  word_vectors.append(model.get_vector(word))
word_vectors = np.asarray(word_vectors)
print(word_vectors[:2])

[[ 0.63006    0.65177    0.25545    0.018593   0.043094   0.047194
   0.23218    0.11613    0.17371    0.40487    0.022524  -0.076731
  -2.2911     0.094127   0.43293    0.041801   0.063175  -0.64486
  -0.43657    0.024114  -0.082989   0.21686   -0.13462   -0.22336
   0.39436   -2.1724    -0.39544    0.16536    0.39438   -0.35182
  -0.14996    0.10502   -0.45937    0.27729    0.8924    -0.042313
  -0.009345   0.55017    0.095521   0.070504  -1.1781     0.013723
   0.17742    0.74142    0.17716    0.038468  -0.31684    0.08941
   0.20557   -0.34328   -0.64303   -0.878     -0.16293   -0.055925
   0.33898    0.60664   -0.2774     0.33626    0.21603   -0.11051
   0.0058673 -0.64757   -0.068222  -0.77414    0.13911   -0.15851
  -0.61885   -0.10192   -0.47       0.19787    0.42175   -0.18458
   0.080581  -0.22545   -0.065129  -0.15328    0.087726  -0.18817
  -0.08371    0.21779    0.97899    0.1092     0.022705  -0.078234
   0.15595    0.083105  -0.6824     0.57469   -0.19942    0.50566
  -0

In [0]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [18]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(n_components=2)
word_vectors_pca = pca.fit_transform(word_vectors)
print(word_vectors_pca)
word_vectors_pca = StandardScaler().fit_transform(word_vectors_pca)
print(word_vectors_pca)
#word_vectors_pca = pd.DataFrame(data = principal_components, columns = ['principal component 1', 'principal component 2'])
#word_vectors_pca = # YOUR CODE

# and maybe MORE OF YOUR CODE here :)

[[ 0.9333849   0.5037116 ]
 [ 0.72930443  0.36441848]
 [ 1.1819206   0.61373425]
 ...
 [ 2.5400124  -2.304001  ]
 [-0.07835428  0.48636347]
 [-2.0053227  -0.22080679]]
[[ 0.38816744  0.29139635]
 [ 0.3032966   0.21081561]
 [ 0.49152586  0.3550442 ]
 ...
 [ 1.0563148  -1.3328588 ]
 [-0.03258405  0.2813605 ]
 [-0.83395153 -0.12773584]]


In [0]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [0]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [41]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1002', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [20]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca

tsne = TSNE(n_components=2, verbose=100)
word_tsne = tsne.fit_transform(word_vectors)


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.005s...
[t-SNE] Computed neighbors for 1000 samples in 0.228s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716134
[t-SNE] Computed conditional probabilities in 0.061s
[t-SNE] Iteration 50: error = 68.9642944, gradient norm = 0.2974837 (50 iterations in 13.588s)
[t-SNE] Iteration 100: error = 69.2145615, gradient norm = 0.2974936 (50 iterations in 11.360s)
[t-SNE] Iteration 150: error = 69.1202011, gradient norm = 0.2785512 (50 iterations in 12.309s)


KeyboardInterrupt: ignored

In [0]:
word_tsne = StandardScaler().fit_transform(word_tsne)

In [0]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [31]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    phrase_lower = phrase.lower()
    # 2. tokenize phrase
    tokens = nltk.word_tokenize(phrase)
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    token_vectors = []
    for token in tokens:
      try: 
        token_vectors.append(model.get_vector(token))
      except:
        token_vectors.append(np.zeros([model.vector_size], dtype='float32'))
    avg_vector = np.mean(token_vectors, axis=0)
    return avg_vector


    # YOUR CODE
    
    #return vector
        
#get_phrase_embedding('What a day!')
get_phrase_embedding("I'm very sure. This never happened to me before...")[::10]
    

array([ 0.26728266, -0.05069667,  0.16060583, -0.00243728, -0.667494  ,
       -0.15167634,  0.17277159,  0.06088366,  0.9743908 , -0.00277224],
      dtype=float32)

In [0]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [0]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = [get_phrase_embedding(x) for x in chosen_phrases]

In [35]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

AssertionError: ignored

In [0]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = StandardScaler().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [42]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1111', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [0]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [0]:
from scipy.spatial.distance import cosine

In [48]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query_vec = get_phrase_embedding(query)
    distances = [cosine(query_vec, i) for i in data_vectors]
    distances = np.argpartition(distances, k)
    results = [data[i] for i in distances]
    return results
find_nearest('Who are you?')

/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


['Who are you? Where did you come from? Where are you going?\n',
 'Why are you Hindu?\n',
 'Where are you?\n',
 'Helo how are you?\n',
 'Dave Consiglio who are you?\n',
 'What are you doing? Why?\n',
 'Which My Little Pony are you?\n',
 'Hi Aune, how are you are there?\n',
 'How are you?\n',
 'How lucky are you?\n',
 'Who are you and what are you?\n',
 'What are you doing?\n',
 'Do you feel you are God?\n',
 'Why are there obelisks?\n',
 'What are cations and anions?\n',
 'What are CT and PT?\n',
 'Wedding91: Who are you?\n',
 'What are you scared of? Why?\n',
 'What are Sensex and Nifty?\n',
 'Why are there Greek Orthodox Churches?\n',
 'Why are you on Tinder?\n',
 'Who are Batman and Robin?\n',
 'What are edX and Coursera?\n',
 'What are CRR and SLR?\n',
 'What are neutrophils and macrophages?\n',
 'What are dendrites and axons?\n',
 'How are you doing?\n',
 'Why are you alive?\n',
 'Where are you from?\n',
 'What are macro-evolution and micro-evolution?\n',
 'How old are you?\n',
 '

In [0]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

In [0]:
find_nearest(query="How does Trump?", k=10)

In [0]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.